In [36]:
import itertools
from collections import defaultdict
import dimod
from dimod import BQM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from neal import SimulatedAnnealingSampler
from dwave.system import DWaveSampler, LeapHybridSampler
from dwave.system import EmbeddingComposite
from IPython.display import display, HTML

In [53]:
def school_bqm(G, L, D, T, classes_amount, time_cost):
    bqm = BQM("BINARY")
    P1 = 10
    P2 = 10
    P3 = 10
    P4 = 10
    P5 = 8
    P6 = 8
    P7 = 3
    P8 = 3
    
    """
    #Add hard constraint (with slack variables)#1
    for g, d, t in itertools.product(G, D, T):
        c1 = [(f"x_{l}_{g}_{d}_{t}", 1) for l in L]
        bqm.add_linear_inequality_constraint(
            c1, lagrange_multiplier=P1, ub=1, label=f"c1_l_{g}_{d}_{t}")
    """
    #Add hard constraint #1
    for g, d, t in itertools.product(G, D, T):
        for l1 in range(len(L)):
            for l2 in range(l1 + 1, len(L)):
                bqm.add_quadratic(f"x_{L[l1]}_{g}_{d}_{t}", f"x_{L[l2]}_{g}_{d}_{t}", P1)
    """
    
    #Add hard constraint (with slack variables)#2
    for l, d, t in itertools.product(L, D, T):
        c2 = [(f"x_{l}_{g}_{d}_{t}", 1) for g in G]
        bqm.add_linear_inequality_constraint(
            c2, lagrange_multiplier=P2, ub=1, label=f"c2_{l}_g_{d}_{t}")
    """
    #Add hard constraint #2
    for l, d, t in itertools.product(L, D, T):
        for g1 in range(len(G)):
            for g2 in range(g1 + 1, len(G)):
                bqm.add_quadratic(f"x_{l}_{G[g1]}_{d}_{t}", f"x_{l}_{G[g2]}_{d}_{t}", P2)
    
    #Add hard constraint #3
    for g, l in itertools.product(G, L):
        c3 = [(f"x_{l}_{g}_{d}_{t}", 1) for d, t in itertools.product(D,T)]
        bqm.add_linear_equality_constraint(c3, lagrange_multiplier=P3, constant=-classes_amount[g][l])
    
    #Add hard constraint #4
    for g, d in itertools.product(G, D):
        c4 = [(f"x_pus_{g}_{d}_{t}", 1) for t in T]
        bqm.add_linear_equality_constraint(c4, lagrange_multiplier=P4, constant=-1)
        
    #Add hard constraint #5
    for g, d, t, l in itertools.product(G, D, T, L):
        bqm.add_quadratic(f"x_pus_{g}_{d}_{t}", f"x_{l}_{g}_{d}_{t}", P4)
        
    #Add hard constraint (with slack variables)#6
    for d, t in itertools.product(D, T):
        c6 = [(f"x_pus_{g}_{d}_{t}", 1) for g in G]
        bqm.add_linear_inequality_constraint(
            c6, lagrange_multiplier=P6, ub=2, label=f"c6_pus_g_{d}_{t}")
        
    #Add soft constraint #1    
    for g, l, d in itertools.product(G, L, D):
        c7 = [(f"x_{l}_{g}_{d}_{t}", 1) for t in T]
        bqm.add_linear_inequality_constraint(
            c7, lagrange_multiplier=P7, ub=2, label=f"c7_{l}_{g}_{d}_t")
    
    #Add soft constraint #2
    for g, d, t in itertools.product(G, D, range(len(T)-1)):
        bqm.add_quadratic(f"x_pus_{g}_{d}_{T[t]}", f"x_Sports_{g}_{d}_{T[t+1]}", P8)
    
    #Add soft constraint #3
    for g, d, t, l in itertools.product(G, D, T, L):
        bqm.add_linear(f"x_{l}_{g}_{d}_{t}", time_cost[g][t])
    for g, d, t in itertools.product(G, D, T):
        bqm.add_linear(f"x_pus_{g}_{d}_{t}", time_cost[g][t])
    
    return bqm

In [38]:
def is_sample_feasible(sample, G, L, D, T, classes_amount):
    
    #Check hard constraint #1
    for g, d, t in itertools.product(G, D, T):
        if sum([sample[f"x_{l}_{g}_{d}_{t}"] for l in L]) > 1:
            return False
    
    #Check hard constraint #2
    for l, d, t in itertools.product(L, D, T):
        if sum([sample[f"x_{l}_{g}_{d}_{t}"] for g in G]) > 1:
            return False
    
    #Check hard constraint #3
    for g, l in itertools.product(G, L):
        if sum([sample[f"x_{l}_{g}_{d}_{t}"] for d, t in itertools.product(D,T)]) != classes_amount[g][l]:
            return False
    
    #Check hard constraint #4
    for g, d in itertools.product(G, D):
        if sum([sample[f"x_pus_{g}_{d}_{t}"] for t in T]) != 1:
            return False    
    
    #Check hard constraint #5
    for g, d, t, l in itertools.product(G, D, T, L):
        if sample[f"x_pus_{g}_{d}_{t}"] * sample[f"x_{l}_{g}_{d}_{t}"] != 0:
            return False 
        
    #Check hard constraint #6
    for d, t in itertools.product(D, T):
        if sum([sample[f"x_pus_{g}_{d}_{t}"] for g in G]) > 2:
            return False
    return True

In [39]:
def get_scheedule(sample, L, D, T, group):
    scheedule = []
    for t in T:
        sc = [t]
        for d in D:
            if sample[f"x_pus_{group}_{d}_{t}"] == 1:
                sc.append("Pusdienas")
            elif sum(sample[f"x_{l}_{group}_{d}_{t}"] for l in L) == 0:
                sc.append("-")
            else:
                for l in L:
                    if sample[f"x_{l}_{group}_{d}_{t}"] == 1:
                        sc.append(l)
        scheedule.append(sc)
    print(group)
    display(HTML(pd.DataFrame(scheedule, columns = ["Time"] + D).to_html()))
    print("-----------------------------------------------")
        

In [40]:
def prepare_big_test():
    G = ["10a", "10b", "11a", "11b", "12a", "12b"]
    L = ["Fizika", "Geografija", "Sports", "LatVal", "Kimija", "Programmesana",
         "Matem1", "Matem2", "AngluVal", "Biologija", "Datorika", "Literatura", "Vesture"]
    D = ["P", "O", "Tr", "C", "Pk"]
    T = [815, 900, 955, 1040, 1145, 1230, 1320, 1410, 1455]
    
    
    ca_10a = {"Fizika": 2, "Geografija": 0, "Sports": 3, "LatVal": 3, "Kimija": 2, "Programmesana": 4,
         "Matem1": 6, "Matem2": 0, "AngluVal": 6, "Biologija": 2, "Datorika": 0, "Literatura": 2, "Vesture": 3}
    ca_10b = {"Fizika": 3, "Geografija": 1, "Sports": 3, "LatVal": 3, "Kimija": 3, "Programmesana": 0,
         "Matem1": 0, "Matem2": 6, "AngluVal": 6, "Biologija": 3, "Datorika": 0, "Literatura": 2, "Vesture": 3}
    ca_11a = {"Fizika": 0, "Geografija": 2, "Sports": 3, "LatVal": 3, "Kimija": 0, "Programmesana": 4,
         "Matem1": 6, "Matem2": 0, "AngluVal": 6, "Biologija": 0, "Datorika": 0, "Literatura": 2, "Vesture": 2}
    ca_11b = {"Fizika": 3, "Geografija": 2, "Sports": 3, "LatVal": 3, "Kimija": 3, "Programmesana": 0,
         "Matem1": 0, "Matem2": 6, "AngluVal": 6, "Biologija": 3, "Datorika": 0, "Literatura": 2, "Vesture": 0}
    ca_12a = {"Fizika": 0, "Geografija": 0, "Sports": 3, "LatVal": 3, "Kimija": 0, "Programmesana": 3,
         "Matem1": 8, "Matem2": 0, "AngluVal": 6, "Biologija": 0, "Datorika": 2, "Literatura": 2, "Vesture": 3}
    ca_12b = {"Fizika": 0, "Geografija": 0, "Sports": 3, "LatVal": 3, "Kimija": 6, "Programmesana": 0,
         "Matem1": 0, "Matem2": 4, "AngluVal": 6, "Biologija": 6, "Datorika": 2, "Literatura": 2, "Vesture": 0}
    classes_amount = {"10a": ca_10a, "10b": ca_10b, "11a": ca_11a, "11b": ca_11b, "12a": ca_12a, "12b": ca_12b}
    
    tc_10a = {815: 0, 900: 0, 955: 0, 1040: 0, 1145: 0, 1230: 0, 1320: 1, 1410: 2, 1455: 3}
    tc_10b = {815: 0, 900: 0, 955: 0, 1040: 0, 1145: 0, 1230: 0, 1320: 1, 1410: 2, 1455: 3}
    tc_11a = {815: 0, 900: 0, 955: 0, 1040: 0, 1145: 0, 1230: 0, 1320: 0, 1410: 1, 1455: 2}
    tc_11b = {815: 0, 900: 0, 955: 0, 1040: 0, 1145: 0, 1230: 0, 1320: 0, 1410: 1, 1455: 2}
    tc_12a = {815: 0, 900: 0, 955: 0, 1040: 0, 1145: 0, 1230: 0, 1320: 0, 1410: 1, 1455: 2}
    tc_12b = {815: 0, 900: 0, 955: 0, 1040: 0, 1145: 0, 1230: 0, 1320: 0, 1410: 1, 1455: 2}
    time_cost = {"10a": tc_10a, "10b": tc_10b, "11a": tc_11a, "11b": tc_11b, "12a": tc_12a, "12b": tc_12b}
    
    return G, L, D, T, classes_amount, time_cost

In [41]:
def prepare_small_test():
    G = ["11a", "11b"]
    L = ["Sports", "LatVal", "Matematika"]
    D = ["P", "O"]
    T = [815, 900, 955, 1040, 1145]
    
    ca_11a = {"Sports": 2, "LatVal": 1, "Matematika": 3}
    ca_11b = {"Sports": 2, "LatVal": 2, "Matematika": 1}
    classes_amount = {"11a": ca_11a, "11b": ca_11b}

    tc_11a = {815: 0, 900: 0, 955: 0, 1040: 0, 1145: 1}
    tc_11b = {815: 0, 900: 0, 955: 0, 1040: 0, 1145: 1}
    time_cost = {"11a": tc_11a, "11b": tc_11b}
    
    return G, L, D, T, classes_amount, time_cost

In [42]:
def prepare_medium_test():
    G = ["11a", "11b", "12a", "12b"]
    L = ["Fizika", "Sports", "LatVal", "Vesture", "Kimija", "MatemOzol", "MatemKaln"]
    D = ["P", "O", "Tr"]
    T = [815, 900, 955, 1040, 1145, 1230, 1320, 1410]
    
    ca_11a = {"Fizika": 2, "Sports": 2, "LatVal": 3, "Vesture": 1, "Kimija": 2, "MatemOzol": 6, "MatemKaln": 0}
    ca_11b = {"Fizika": 2, "Sports": 2, "LatVal": 3, "Vesture": 1, "Kimija": 2, "MatemOzol": 3, "MatemKaln": 0}
    ca_12a = {"Fizika": 2, "Sports": 2, "LatVal": 3, "Vesture": 1, "Kimija": 2, "MatemOzol": 0, "MatemKaln": 6}
    ca_12b = {"Fizika": 2, "Sports": 2, "LatVal": 3, "Vesture": 1, "Kimija": 2, "MatemOzol": 0, "MatemKaln": 3}
    classes_amount = {"11a": ca_11a, "11b": ca_11b, "12a": ca_12a, "12b": ca_12b}

    tc_11a = {815: 0, 900: 0, 955: 0, 1040: 0, 1145: 0, 1230: 0, 1320: 1, 1410: 2}
    tc_11b = {815: 0, 900: 0, 955: 0, 1040: 0, 1145: 0, 1230: 0, 1320: 1, 1410: 2}
    tc_12a = {815: 0, 900: 0, 955: 0, 1040: 0, 1145: 0, 1230: 0, 1320: 1, 1410: 2}
    tc_12b = {815: 0, 900: 0, 955: 0, 1040: 0, 1145: 0, 1230: 0, 1320: 1, 1410: 2}
    time_cost = {"11a": tc_11a, "11b": tc_11b, "12a": tc_12a, "12b": tc_12b}
    
    return G, L, D, T, classes_amount, time_cost

In [70]:
G, L, D, T, classes_amount, time_cost = prepare_big_test()
#G, L, D, T, classes_amount, time_cost = prepare_medium_test()
#G, L, D, T, classes_amount, time_cost = prepare_small_test()
bqm = school_bqm(G, L, D, T, classes_amount, time_cost)
len(bqm.variables)

4650

## Simulated Annealing

Can handle problem instances of any size

In [75]:
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads = 350)

print(sampleset)

    slack_c6_pus_g_C_1040_0 ... x_pus_12b_Tr_955 energy num_oc.
7                         0 ...                0   68.0       1
229                       0 ...                0   69.0       1
10                        0 ...                0   71.0       1
83                        0 ...                0   72.0       1
159                       0 ...                0   72.0       1
186                       1 ...                0   72.0       1
272                       0 ...                0   72.0       1
306                       1 ...                0   72.0       1
52                        0 ...                0   74.0       1
60                        1 ...                0   74.0       1
207                       1 ...                0   74.0       1
280                       1 ...                0   74.0       1
16                        1 ...                0   75.0       1
99                        0 ...                0   75.0       1
131                       1 ...         

In [76]:
best_sample = sampleset.first.sample
is_feasible = is_sample_feasible(best_sample, G, L, D, T, classes_amount)
print(is_feasible)
for g in G:
    get_scheedule(best_sample, L, D, T, g)

True
10a


,Time,P,O,Tr,C,Pk
0,815,AngluVal,Matem1,Matem1,LatVal,AngluVal
1,900,LatVal,Kimija,Programmesana,Sports,Vesture
2,955,-,AngluVal,Pusdienas,Matem1,Matem1
3,1040,Pusdienas,Programmesana,Fizika,Biologija,Kimija
4,1145,Vesture,Literatura,LatVal,AngluVal,Fizika
5,1230,-,Pusdienas,AngluVal,Pusdienas,Pusdienas
6,1320,Biologija,Matem1,Sports,AngluVal,-
7,1410,Matem1,Literatura,Sports,-,Programmesana
8,1455,Programmesana,-,-,Vesture,-


-----------------------------------------------
10b


,Time,P,O,Tr,C,Pk
0,815,Geografija,AngluVal,Literatura,Matem2,Sports
1,900,Matem2,LatVal,Biologija,Vesture,Pusdienas
2,955,Pusdienas,Vesture,Matem2,AngluVal,Biologija
3,1040,-,Matem2,-,Pusdienas,AngluVal
4,1145,Matem2,Kimija,Pusdienas,Fizika,LatVal
5,1230,Fizika,Sports,Kimija,LatVal,Fizika
6,1320,Vesture,-,Biologija,Literatura,AngluVal
7,1410,Sports,Pusdienas,AngluVal,Kimija,Matem2
8,1455,-,-,AngluVal,-,-


-----------------------------------------------
11a


...

-----------------------------------------------
12a


,Time,P,O,Tr,C,Pk
0,815,-,Datorika,Sports,AngluVal,-
1,900,Sports,Programmesana,Pusdienas,Matem1,LatVal
2,955,Vesture,Programmesana,Matem1,-,Programmesana
3,1040,Datorika,Matem1,AngluVal,Pusdienas,Matem1
4,1145,Pusdienas,-,AngluVal,LatVal,Pusdienas
5,1230,Literatura,-,LatVal,Matem1,Vesture
6,1320,Matem1,Pusdienas,Matem1,Vesture,-
7,1410,-,Matem1,-,AngluVal,Sports
8,1455,AngluVal,AngluVal,Literatura,-,-


-----------------------------------------------
12b


,Time,P,O,Tr,C,Pk
0,815,-,Pusdienas,Kimija,-,Kimija
1,900,Pusdienas,AngluVal,Pusdienas,Biologija,AngluVal
2,955,LatVal,-,Literatura,Matem2,-
3,1040,Kimija,LatVal,Biologija,AngluVal,Biologija
4,1145,AngluVal,Matem2,Biologija,Kimija,Kimija
5,1230,AngluVal,Biologija,Sports,Datorika,Pusdienas
6,1320,Datorika,Kimija,AngluVal,Pusdienas,-
7,1410,Biologija,Sports,-,Matem2,-
8,1455,Sports,-,LatVal,Literatura,Matem2


-----------------------------------------------


## Quantum Annealing
First of all we create quantum and hybid solvers, by providing Leap subscription token

In [45]:
token_val = "DEV-fc61310ff217929feac5a5ec87b1b5e23302c79a"
qpu_advantage = EmbeddingComposite(DWaveSampler(solver={'topology__type': 'pegasus'}, token=token_val))
hybrid_bqm_sampler = LeapHybridSampler(token=token_val)

### Hybrid solver
Can be used for problem instances of any size

In [71]:
hybrid_sampleset = hybrid_bqm_sampler.sample(bqm)
print(hybrid_sampleset)

  slack_c6_pus_g_C_1040_0 ... x_pus_12b_Tr_955 energy num_oc.
0                       1 ...                0   52.0       1
['BINARY', 1 rows, 1 samples, 4650 variables]


In [72]:
best_hybrid_sample = hybrid_sampleset.first.sample
is_feasible = is_sample_feasible(best_hybrid_sample, G, L, D, T, classes_amount)
print(is_feasible)
for g in G:
    get_scheedule(best_hybrid_sample, L, D, T, g)

True
10a


,Time,P,O,Tr,C,Pk
0,815,Biologija,AngluVal,Biologija,Kimija,Pusdienas
1,900,AngluVal,Sports,AngluVal,Programmesana,Matem1
2,955,Programmesana,Sports,Fizika,Matem1,Literatura
3,1040,Kimija,Fizika,Literatura,Vesture,AngluVal
4,1145,LatVal,Programmesana,Sports,Pusdienas,Vesture
5,1230,Matem1,AngluVal,Matem1,AngluVal,Programmesana
6,1320,Pusdienas,Pusdienas,Pusdienas,Matem1,Vesture
7,1410,Matem1,LatVal,-,-,-
8,1455,-,-,-,-,LatVal


-----------------------------------------------
10b


,Time,P,O,Tr,C,Pk
0,815,Vesture,-,Vesture,Fizika,Fizika
1,900,Pusdienas,-,Matem2,AngluVal,Sports
2,955,AngluVal,Biologija,Matem2,Matem2,Pusdienas
3,1040,LatVal,LatVal,AngluVal,Matem2,Fizika
4,1145,Biologija,Biologija,-,AngluVal,Literatura
5,1230,Matem2,Sports,Pusdienas,Pusdienas,Kimija
6,1320,Geografija,Pusdienas,Vesture,LatVal,Matem2
7,1410,Kimija,AngluVal,Sports,Kimija,Literatura
8,1455,-,-,-,-,AngluVal


-----------------------------------------------
11a


...

-----------------------------------------------
12a


,Time,P,O,Tr,C,Pk
0,815,Matem1,Datorika,-,AngluVal,Programmesana
1,900,LatVal,Matem1,Literatura,LatVal,-
2,955,Vesture,Vesture,-,Sports,Sports
3,1040,Programmesana,Pusdienas,-,Programmesana,Matem1
4,1145,Matem1,AngluVal,Pusdienas,Matem1,Matem1
5,1230,Vesture,LatVal,Datorika,Pusdienas,Pusdienas
6,1320,Literatura,Sports,AngluVal,-,-
7,1410,Pusdienas,Matem1,Matem1,AngluVal,AngluVal
8,1455,AngluVal,-,-,-,-


-----------------------------------------------
12b


,Time,P,O,Tr,C,Pk
0,815,LatVal,Sports,Kimija,-,Pusdienas
1,900,Biologija,Kimija,Kimija,Biologija,Biologija
2,955,Literatura,Matem2,Biologija,AngluVal,AngluVal
3,1040,Matem2,Sports,Datorika,LatVal,Literatura
4,1145,Pusdienas,Pusdienas,AngluVal,Sports,LatVal
5,1230,Kimija,Datorika,Pusdienas,Biologija,Matem2
6,1320,Kimija,AngluVal,-,-,-
7,1410,AngluVal,Kimija,Matem2,Pusdienas,Biologija
8,1455,-,-,-,AngluVal,-


-----------------------------------------------


### Quantum solver
Can be used only for small problem instances with size of ~100 variables 

In [58]:
quantum_sampleset = qpu_advantage.sample(bqm, num_reads = 4000)
print(quantum_sampleset)

     slack_c7_LatVal_11a_O_t_0 ... x_pus_11b_P_955 energy num_oc. chain_b.
410                          0 ...               0   11.0       1 0.028846
0                            0 ...               0   21.0       1      0.0
29                           1 ...               0   21.0       1 0.009615
148                          0 ...               0   21.0       1 0.028846
8                            1 ...               1   24.0       1 0.009615
286                          0 ...               0   24.0       1 0.019231
1                            0 ...               0   25.0       1      0.0
270                          1 ...               0   26.0       1 0.009615
2                            1 ...               0   28.0       1      0.0
81                           0 ...               1   28.0       1 0.019231
406                          0 ...               0   28.0       1 0.019231
3                            0 ...               0   29.0       1      0.0
80                       

In [59]:
best_quantum_sample = quantum_sampleset.first.sample
is_feasible = is_sample_feasible(best_quantum_sample, G, L, D, T, classes_amount)
print(is_feasible)
for g in G:
    get_scheedule(best_quantum_sample, L, D, T, g)

True
11a


,Time,P,O
0,815,Matematika,-
1,900,Pusdienas,Pusdienas
2,955,Sports,Matematika
3,1040,Sports,Matematika
4,1145,-,LatVal


-----------------------------------------------
11b


,Time,P,O
0,815,Pusdienas,Matematika
1,900,LatVal,Sports
2,955,-,LatVal
3,1040,-,Sports
4,1145,-,Pusdienas


-----------------------------------------------
